In [1]:
def list_split(X, index, feature, split):
    split_points = [[], []]
    while index:
        if X[index[0]][feature] < split:
            split_points[0].append(index.pop(0))
        else:
            split_points[1].append(index.pop(0))
    return split_points

In [2]:
class Node(object):
    def __init__(self,score=None):
        self.score = score
        self.left = None
        self.right = None
        self.feature = None
        self.split = None
        
class RegressionTree(object):
    def __init__(self):
        self.root = Node()
        self.depth = 1
        self._rules = None
        
    def getMSE(self,X,y,index,feature,split):
        # left node, right node
        split_sum = [0,0]
        split_cnt = [0,0]
        split_sqr_sum = [0,0]
        
        for i in index:
            xi,yi = X[i][feature],y[i]
            if xi < split:
                split_cnt[0]+=1
                split_sum[0]+=yi
                split_sqr_sum[0]+=yi**2
            else:
                split_cnt[1]+=1
                split_sum[1]+=yi
                split_sqr_sum[1]+=yi**2
        
        avg = [split_sum[0]/split_cnt[0],split_sum[1]/split_cnt[1]]
        mse = [(split_sqr_sum[0]**2+2*avg[0]*split_sum[0]+avg[0]**2)/split_cnt[0],
              (split_sqr_sum[1]**2+2*avg[1]*split_sum[1]+avg[1]**2)/split_cnt[1]]
        return sum(mse),split,avg
    
    def chooseSplit(self,X,y,index,feature):
        unique = set([X[i][feature] for i in index])
        if len(unique)==1:
            return None
        unique.remove(min(unique)) #prevent split_cnt is qeual to 0; if you didn't remove the least, you will get a divisionByZeroError
        mse,split,avg = min(
            [self.getMSE(X,y,index,feature,split)for split in unique],
            key=lambda x:x[0])
        return mse,feature,split,avg 
    
    def chooseFeature(self,X,y,index):
        number_of_feature = len(X[0])
        split_points = map(lambda fs:self.chooseSplit(X,y,index,fs),range(number_of_feature))
        split_points = filter(lambda sp:sp is not None, split_points)
        min_mse,min_feature,min_split,min_avg = min(split_points,key=lambda x:x[0])
        return min_mse,min_feature,min_split,min_avg
    
    def getRules(self):
        queue = [[self.root,[]]]
        self._rules=[]
        while queue:
            node,rule = queue.pop(0)
            if not(node.left or node.right):
                self._rules.append([node.score])
            if node.left:
                rule_left = rule[:]
                rule_left.append([node.feature,-1,node.split])
                queue.append([node.left,rule_left])
            if node.right:
                rule_right = rule[:]
                rule_right.append([node.feature,1,node.split])
                queue.append([node.right, rule_right])
                
    def fit(self,X,y,max_depth=6,min_samples_split=2):
        self.root.score = sum(y)/len(y)
        idx = list(range(len(y)))
        queue = [(self.depth+1,self.root,idx)]
        
        while queue:
            depth,node,index = queue.pop(0)
            if depth>max_depth:
                depth-=1
                break
                
            if len(index)<min_samples_split or len(set(y[index]))==1:
                continue
            
            split_points = self.chooseFeature(X,y,index)
            if split_points is None:
                continue
                
            _,feature,split,avg = split_points
            node.feature = feature
            node.split = split
            node.left = Node(avg[0])
            node.right = Node(avg[1])
            
            idx_split = list_split(X,index,feature,split)
            queue.append((depth+1,node.left,idx_split[0]))
            queue.append((depth+1,node.right,idx_split[1]))
        self.depth = depth
        self.getRules()
        
    @property
    def rules(self):
        for i,rule in enumerate(self._rules):
            score = rule
            print('Rule %d, y_hat %.4f'%(i,score))
            
    def _predict(self,Xi):
        node = self.root
        while node.left and node.right:
            if Xi[node.feature] < node.split:
                node = node.left
            else:
                node = node.right
        return node.score
    
    def predict(self,X):
        return [self._predict(Xi) for Xi in X]

In [5]:
def loss(model, X, y):
    y_pred = model.predict(X)
    m = len(y)
    n = len(y_pred)
    y_avg = sum(y) / len(y)
    r2 = 1 - sum((yi - yi_pred) ** 2 for yi, yi_pred in zip(y, y_pred)) / sum((yi - y_avg) ** 2 for yi in y)
    print("Test r2 is %.3f!" % r2)
    return r2

In [6]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
X,y = load_boston()['data'],load_boston()['target']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=123)
RT = RegressionTree()
RT.fit(X=X_train,y=y_train,max_depth=5)
loss(RT,X_test,y_test)

Test r2 is 0.033!


0.03278921810057589